In [1]:
import os
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import DesiredCapabilities
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--window-size=1280,1696')
#chrome_options.add_argument('--start-maximized')

driver = webdriver.Chrome(executable_path=os.path.abspath('chromedriver'), options=chrome_options,
                            desired_capabilities=capabilities)

driver.get('http://10.1.5.82/ocsreports/?function=visu_computers')
login = driver.find_element_by_id('LOGIN').send_keys('admin')
senha = driver.find_element_by_id('PASSWD')
senha.send_keys('admin')
senha.send_keys(Keys.ENTER)

time.sleep(1)

# Seleciona todas a colunas
driver.find_element_by_xpath('/html/body/div/form[1]/div[2]/div/div/div[1]/div[1]/label/select/option[7]').click()
time.sleep(1)

# Pega toda a tabela
table_id = driver.find_element(By.XPATH, '//*[@id="list_show_all"]/tbody')

rows = table_id.find_elements(By.TAG_NAME, 'tr')

In [3]:
listaHyperlink = []

for row in rows:

    a = row.find_element_by_tag_name('a')
    #nome = row.find_element_by_tag_name('a').text
    hyperlink = a.get_attribute('href')
    listaHyperlink.append(hyperlink)
    
    

In [4]:
list_ativo = []
list_desc = []

for link in listaHyperlink:
    driver.get(link)
    driver.find_element_by_xpath('/html/body/div/div[1]/ul/li[2]/a').click()
    storage = driver.find_element(By.XPATH, '//*[@id="affich_drives"]/tbody')
    rowst = storage.find_elements(By.TAG_NAME, "tr")

    for row in rowst:
        ativo = driver.find_element_by_xpath('/html/body/div/div[2]/h3')
        list_ativo.append(ativo.text)
        list_desc.append(row.text)
        
driver.quit()

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame({'nome': list_ativo, 'desc': list_desc})

In [ ]:
df['hd'] = df.desc.str[0:3]

In [ ]:
df = df[(df.nome.str.contains('SR|CF|CLF'))&(df.desc.str.contains('NTFS'))]

In [ ]:
df['desc'] = df.desc.str.replace('\w: ','').str.replace(' NTFS ', '|NTFS|').str.replace('\n\d\d?%', '').str.split('|')

In [ ]:
df['hd_desc'] = df.desc.str[0]

In [122]:
df['free_space'] = df.desc.str[2].str.split(' ').str[0]

In [123]:
df['total_space'] = df.desc.str[2].str.split(' ').str[1]

In [124]:
df.drop(columns='desc', inplace=True)
df.reset_index(drop=True, inplace=True)

In [125]:
df['tipo'] = df.nome.apply(
    lambda x: 'Desktop' if x[0:1]=='D' else x).apply(
    lambda x: 'Notebook' if x[0:1]=='N' else x).apply(
    lambda x: 'Servidor' if x[0:1]=='C' else x).apply(
    lambda x: 'Servidor' if x[0:1]=='S' else x)    

In [126]:
df[['free_space', 'total_space']] = df[['free_space', 'total_space']].apply(pd.to_numeric)

In [127]:
df['used_space'] = df.total_space-df.free_space

In [128]:
df = df[['tipo', 'nome', 'hd', 'hd_desc', 'used_space', 'free_space', 'total_space']]

In [129]:
df['used_space'] = round(df.used_space/1024,4)
df['free_space'] = round(df.free_space/1024,4)
df['total_space'] = round(df.total_space/1024,4)
df['%_used'] = round(df.used_space/df.total_space,4)
df['%_free'] = round(df.free_space/df.total_space,4)
df.sort_values(by=['tipo', 'nome'], inplace=True)
df.to_csv('X:\TI\Documentos\Relatorio_Espaço_em_disco\Relatorio\R_Espaço_em_disco.csv', index=False)

In [130]:
df

,tipo,nome,hd,hd_desc,used_space,free_space,total_space,%_used,%_free
16,Desktop,DKCLFADM01,C:,Hard Drive OS,31.4580,898.3145,929.7725,0.0338,0.9662
9,Desktop,DKCLFALM01,C:,Hard Drive,52.9873,412.1719,465.1592,0.1139,0.8861
56,Desktop,DKCLFCNC01,C:,Hard Drive,129.5742,93.3945,222.9688,0.5811,0.4189
96,Desktop,DKCLFCNC02,C:,Hard Drive,85.5430,137.3955,222.9385,0.3837,0.6163
85,Desktop,DKCLFCNC03,C:,Hard Drive,88.1279,134.8105,222.9385,0.3953,0.6047
...,...,...,...,...,...,...,...,...,...
42,Servidor,SVCLFDTC02DC02,C:,Hard Drive SO,30.3643,19.0332,49.3975,0.6147,0.3853
38,Servidor,SVCLFDTC02HV01,C:,Hard Drive,37.1504,240.7109,277.8613,0.1337,0.8663
39,Servidor,SVCLFDTC02HV02,C:,Hard Drive,199.5850,78.2773,277.8623,0.7183,0.2817
40,Servidor,SVCLFDTC02HV02,Q:,Hard Drive QUORUM,0.0625,0.9336,0.9961,0.0627,0.9373
